## 使用vits模拟声音做中文TTS

In [1]:
!git clone https://github.com/xhb/vits_tts_zh.git --recursive

Cloning into 'vits_tts_zh'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 7 (delta 0), reused 7 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), 4.44 KiB | 4.44 MiB/s, done.
Submodule 'vits' (https://github.com/zixiiu/vits) registered for path 'vits'
Cloning into '/content/vits_tts_zh/vits'...
remote: Enumerating objects: 633, done.        
remote: Counting objects: 100% (6/6), done.        
remote: Compressing objects: 100% (6/6), done.        
remote: Total 633 (delta 0), reused 2 (delta 0), pack-reused 627        
Receiving objects: 100% (633/633), 31.30 MiB | 11.05 MiB/s, done.
Resolving deltas: 100% (300/300), done.
Submodule path 'vits': checked out 'edc91484e8abf60a2a689f36110f983b55f0bf8b'


## 下载vits模型

In [2]:
%cd /content/vits_tts_zh/

/content/vits_tts_zh


In [3]:
!python download-model.py xiier/vits_zh

[{'type': 'file', 'oid': 'c7d9f3332a950355d5a77d85000f05e6f45435ea', 'size': 1477, 'path': '.gitattributes'}, {'type': 'file', 'oid': '86b0a7ce4ee40c498d524daff28679cbcf7ee233', 'size': 211, 'path': 'README.md'}, {'type': 'file', 'oid': '2a96295bd7b87d42b778a2cce12ff0536cb5bc03', 'size': 1830, 'path': 'miko.json'}, {'type': 'file', 'oid': '57c235de63c577b91c8710d417afd308bb9d68c8', 'size': 449983387, 'lfs': {'oid': 'f71962de5a05a61fc51d9f12af981d3d4912701130eff6d832a9d41df8f92885', 'size': 449983387, 'pointerSize': 134}, 'path': 'miko_139k.pth'}, {'type': 'file', 'oid': '4783c98b1b12a970e1cb03af7a1d217329cdeb63', 'size': 449979566, 'lfs': {'oid': 'd9cd73817c59344a24426d0d4a9c2804f1e7d28bb3f462b3417127d10222cb66', 'size': 449979566, 'pointerSize': 134}, 'path': 'miko_90k.pth'}, {'type': 'file', 'oid': '2a8b4126d689aa1aef0a16b3f69766fad5139909', 'size': 1842, 'path': 'paimon6k.json'}, {'type': 'file', 'oid': 'bf808fe6ce126b13c7d77d73ad2ac979b3ff5d6e', 'size': 449983387, 'lfs': {'oid': 'b

### 安装依赖库

In [4]:
!pip install -r ./vits/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.2/974.2 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 25.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 82.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

### 构建vits的monotonic_align动态库

In [5]:
%cd /content/vits_tts_zh/vits/monotonic_align
!mkdir monotonic_align
!python setup.py build_ext --inplace

/content/vits_tts_zh/vits/monotonic_align
Compiling core.pyx because it changed.
[1/1] Cythonizing core.pyx
/usr/local/lib/python3.10/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/vits_tts_zh/vits/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
core.c: In function ‘__Pyx_InitGlobals’:
core.c:16766:1: warning: ‘PyEval_InitThreads’ is deprecated [-Wdeprecated-declarations]
16766 | PyEval_InitThreads();
      | ^~~~~~~~~~~~~~~~~~
In file included from /usr/include/python3.10/Python.h:130,
                 from core.c:16:
/usr/include/python3.10/ceval.h:122:37: note: declared here
  122 | Py_DEPRECATED(3.9) PyAPI_FUNC(void) PyEval_InitThreads(void);
      |                                     ^~~~~~~~~~~~~~~~~~


In [6]:
%cd /content/vits_tts_zh/

/content/vits_tts_zh


### 使用gradio构建测试界面

In [7]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 830.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.1/288.1 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:

In [9]:
import tempfile
import gradio as gr
from TTService import TTService

voices = {
  '派蒙': ['models/xiier_vits_zh/paimon6k.json', 'models/xiier_vits_zh/paimon6k_390k.pth', 'character_paimon', 1],
  '云飞': ['models/xiier_vits_zh/yunfeimix2.json', 'models/xiier_vits_zh/yunfeimix2_53k.pth', 'character_yunfei', 1],
  '神子': ['models/xiier_vits_zh/miko.json', 'models/xiier_vits_zh/miko_139k.pth', 'character_miko', 1],
  '钟离': ['models/xiier_vits_zh/zhongli.json', 'models/xiier_vits_zh/zhongli_44k.pth', 'character_zhongli', 1],
}

characters = list(voices.keys())

def transcribe(text: str, character: str):
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as fp:
        tts = TTService(*voices[character])
        tts.read_save(text, fp, tts.hps.data.sampling_rate)
        return fp.name

inputs = [
    gr.Textbox(label="输入文字", value="", max_lines=3), 
    gr.Radio(label="角色", choices=characters, value="派蒙")
]

outputs = gr.Audio(label="输出语音")

demo = gr.Interface(fn=transcribe, inputs=inputs, outputs=outputs)

demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>